In [33]:
import json
import operator
from sets import Set
import math

checkins = sc.textFile('../YelpDataset/yelp_academic_dataset_checkin.json')
businesses = sc.textFile('../YelpDataset/yelp_academic_dataset_business.json')
tips = sc.textFile('../YelpDataset/yelp_academic_dataset_tip.json')
users = sc.textFile('../YelpDataset/yelp_academic_dataset_user.json')
reviews = sc.textFile('../YelpDataset/yelp_academic_dataset_review.json')
positive_words = sc.textFile('positive-words.txt').collect()
negative_words = sc.textFile('negative-words.txt').collect()
negative_words = negative_words[0].split(",")
positive_words = positive_words[0].split(",")

negative_words = Set(negative_words)
positive_words = Set(positive_words)

selectedCategories = ['Restaurants', 'Shopping', 'Beauty & Spas', 'Bars', 'Automotive', 'Event Planning & Services',
                     'Fast Food', 'Coffee & Tea', 'Hotels', 'Real Estate', 'Dentists', 'Gyms']

print math.log(1)

0.0


In [34]:
# Reviews

def mapper_reviews_filter(rows):
    for row in rows:
        row = json.loads(row)
        yield (row['user_id'], (row['business_id'], row['stars']))

def mapper_business_filter(rows):
    for row in rows:
        row = json.loads(row)
        for category in row['categories']:
            if(category in selectedCategories):
                yield (row['business_id'], category)

def mapper_users_filter(rows):
    for row in rows:
        row = json.loads(row)
        elite = 0
        if len(row['elite']) == 0:
            elite = 1
        else:
            elite = 4 + len(row['elite'])
        yield (row['user_id'], elite)

def mapper_business_rating(rows):
    for row in rows:
        business_id = row[0]
        ratings_value = 0
        ratings_count = 0
        for rating in row[1]:
            ratings_value += rating[0]*rating[1]
            ratings_count += rating[1]
        if(ratings_value < 0):
            rating_value = 0
        yield (business_id, (float(ratings_value)/float(ratings_count), ratings_count))        

def mapper_normalized_total_reviews(records):
    for category in records:
        maxRatingValue = math.log(max(category[1],key=operator.itemgetter(1))[1][0]) 
        minRatingValue = math.log(min(category[1],key=operator.itemgetter(1))[1][0]) 
        maxRatingCount = math.log(max(category[1],key=lambda item:item[1][1])[1][1])
        minRatingCount = math.log(min(category[1],key=lambda item:item[1][1])[1][1])
        for business in category[1]:
            normalizedRatingValue = (math.log(float(business[1][0])) - minRatingValue)/(maxRatingValue - minRatingValue) * 65
            normalizedRatingCount = (math.log(float(business[1][1])) - minRatingCount)/(maxRatingCount - minRatingCount) * 5
            yield ((category[0], business[0]), normalizedRatingValue + normalizedRatingCount)   
            
reviews_filtered = reviews.mapPartitions(mapper_reviews_filter)
business_filtered = businesses.mapPartitions(mapper_business_filter)
users_filtered = users.mapPartitions(mapper_users_filter)

joined_reviews = reviews_filtered.join(users_filtered).map(lambda x: (x[1][0][0],(x[1][0][1], x[1][1]))).groupByKey()
total_reviews = joined_reviews.mapPartitions(mapper_business_rating)
category_join = total_reviews.join(business_filtered).map(lambda x: (x[1][1], (x[0], x[1][0]))).groupByKey().mapValues(list)
total_normalized_reviews = category_join.mapPartitions(mapper_normalized_total_reviews)
# total_sortedByRatingsCount = total_normalized_reviews.map(lambda (k, v): (k, sorted(v, key=lambda x: x[1][1], reverse=True)))
# total_sortedByRatingsValue = total_normalized.map(lambda (k, v): (k, sorted(v, key=lambda x: x[1], reverse=True)))
total_normalized_reviews.take(10)


[((u'Restaurants', u'sOcZcXcNm8LmdoOYqEDqpg'), 60.889657211172825),
 ((u'Restaurants', u'51qNc8sg9kCD5GEJ3TUrrw'), 50.95513110558408),
 ((u'Restaurants', u'Yl7jil0y_1GK1Z82vmUQew'), 48.68231001856046),
 ((u'Restaurants', u'VZYMInkjRJVHwXVFqeoMWg'), 48.69483975770998),
 ((u'Restaurants', u'bZX-b9fInWIk8j3Y50uBWQ'), 51.55066863538831),
 ((u'Restaurants', u'DeiWR9aL18WpVyzuey7dGg'), 59.79133280292059),
 ((u'Restaurants', u'fvzr5GVCnNpnF9V-MvYrFg'), 52.60608576647458),
 ((u'Restaurants', u'JL5B_s1EoN4zIru4JSw4Qg'), 49.15831028012707),
 ((u'Restaurants', u'dzZnC5YVzZVzW0he28HwEA'), 0.0),
 ((u'Restaurants', u'83C1eqPfmQHTy_8MC4Ug6w'), 47.78569825261467)]

In [35]:
# Tips

def mapper_tips_filter(records):
    for row in records:
        row = json.loads(row)
        if (row["type"] == 'tip'):
            text = row["text"].split(" ")
            pos_counter = 0
            neg_counter = 0
            for i in text:
                if i in positive_words:
                    pos_counter += 1
                elif i in negative_words:
                    neg_counter += 1
            if pos_counter > neg_counter:
                yield (row["user_id"], (row["business_id"], 1))  #good
            else:
                yield (row["user_id"], (row["business_id"], -1)) #bad tip

def mapper_normalized_total_tips(records):
    for category in records:
        maxRatingValue = max(category[1],key=operator.itemgetter(1))[1][0]
        minRatingValue = min(category[1],key=operator.itemgetter(1))[1][0]
        maxRatingCount = max(category[1],key=lambda item:item[1][1])[1][1]
        minRatingCount = min(category[1],key=lambda item:item[1][1])[1][1]
        if minRatingValue <= 0:
            minRatingValue = 1
        if maxRatingValue <= 0:
            maxRatingValue = 1
        if minRatingCount <= 0:
            minRatingCount = 1
        if maxRatingCount <= 0:
            maxRatingCount = 1
        minRatingValueLog = math.log(float(minRatingValue))
        maxRatingValueLog = math.log(float(maxRatingValue))
        minRatingCountLog = math.log(float(minRatingCount))
        maxRatingCountLog = math.log(float(maxRatingCount))
        for business in category[1]:
            if(float(business[1][0]) <= 0):
                normalizedRatingValue = 1
            else:
                if(maxRatingValueLog - minRatingValueLog == 0):
                    normalizedRatingValue = 0
                else: 
                    normalizedRatingValue = (math.log(float(business[1][0])) - minRatingValue)/(maxRatingValue - minRatingValue) * 9
            if(float(business[1][1]) <= 0):
                normalizedRatingCount = 1
            else:
                if(maxRatingCountLog - minRatingCountLog):
                    normalizedRatingCount = 0
                else:
                    normalizedRatingCount = (math.log(float(business[1][1])) - minRatingCount)/(maxRatingCount - minRatingCount) * 1           
            yield ((category[0], business[0]), normalizedRatingValue + normalizedRatingCount)   
                
tips_filtered = tips.mapPartitions(mapper_tips_filter)
joined_tips = tips_filtered.join(users_filtered).map(lambda x: (x[1][0][0],(x[1][0][1], x[1][1]))).groupByKey()
total_tips = joined_tips.mapPartitions(mapper_business_rating)
category_join_tips = total_tips.join(business_filtered).map(lambda x: (x[1][1], (x[0], x[1][0]))).groupByKey().mapValues(list)
total_normalized_tips = category_join_tips.mapPartitions(mapper_normalized_total_tips)
total_normalized_tips.take(10)

[((u'Bars', u'nx8B4SQC4pQnMoTVsMs0rg'), 1),
 ((u'Bars', u'XKwbA7WVROVbLlPPzHZHgw'), 1),
 ((u'Bars', u'W-UBevF6tFgX4WiA8PvdPA'), 1),
 ((u'Bars', u'oqL1hS934CyZc3msupQNRA'), 1),
 ((u'Bars', u'D3a0vrdXzT_UkbPNhK8f7w'), 0),
 ((u'Bars', u'g8dcjPzIJGwY-u4GAeHxnQ'), 1),
 ((u'Bars', u'MnHlTUSAPev7YRbdZDv5ew'), 1),
 ((u'Bars', u'cexR4j7jfgnTMbbb_YEDWg'), 1),
 ((u'Bars', u'CBoMK_j_QG-DdMIsqznTkQ'), 1),
 ((u'Bars', u'-SB5exmdSotbAFh-NwMmbQ'), 1)]

In [72]:
#Checkins

def mapper_checkins_filter(records):
    for line in records:
        line = json.loads(line)
        checkin_info = line['checkin_info']
        for key, val in checkin_info.iteritems():
            yield (line['business_id'], val)

def mapper_normalization(records):
    for category in records:
        minCheckins = min(category[1], key=operator.itemgetter(1))[1]
        maxCheckins = max(category[1], key=operator.itemgetter(1))[1]
        if(minCheckins <= 0):
            minCheckins = 1
        if(maxCheckins <= 0):
            maxCheckins = 1
        minCheckins = math.log(minCheckins)
        maxCheckins = math.log(maxCheckins)
        for business in category[1]:
            score = float(business[1])
            if(score <= 0):
                score = 1
            normalizedRatingValue = (math.log(score) - minCheckins)/(maxCheckins - minCheckins) * 20
            yield ((category[0], business[0]), normalizedRatingValue)    
        
checkins_filtered = checkins.mapPartitions(mapper_checkins_filter).reduceByKey(operator.add)
joined_checkins = checkins_filtered.join(business_filtered).map(lambda x: (x[1][1],(x[0],x[1][0]))).groupByKey().mapValues(list)
total_normalized_checkins = joined_checkins.mapPartitions(mapper_normalization)
total_normalized_checkins.take(20)

[(u'cE27W9VPgO88Qxe4ol6y_g', 1),
 (u'cE27W9VPgO88Qxe4ol6y_g', 1),
 (u'cE27W9VPgO88Qxe4ol6y_g', 1),
 (u'cE27W9VPgO88Qxe4ol6y_g', 1),
 (u'cE27W9VPgO88Qxe4ol6y_g', 1),
 (u'cE27W9VPgO88Qxe4ol6y_g', 1),
 (u'cE27W9VPgO88Qxe4ol6y_g', 1),
 (u'cE27W9VPgO88Qxe4ol6y_g', 1),
 (u'cE27W9VPgO88Qxe4ol6y_g', 1),
 (u'mVHrayjG3uZ_RLHkLj-AMg', 1)]

In [61]:
def mapper_business_information(rows):
    for row in rows:
        row = json.loads(row)
        yield(row['business_id'], (row['name'], row['city'], row['latitude'], row['longitude']))

score = total_normalized_checkins.join(total_normalized_reviews).join(total_normalized_tips)
score = score.map(lambda x: (x[0][1], ((x[0][0], (x[1][0][0], x[1][0][1], x[1][1]), x[1][0][0] + x[1][0][1] + x[1][1]))))
business_information = businesses.mapPartitions(mapper_business_information)
business_information_joined = business_information.join(score)
business_information_joined.take(2)

[(u'OhmRdo9XL2ZWtpt7ewvPAg',
  ((u'Subway', u'Henderson', 36.0288225, -115.0624726),
   (u'Restaurants',
    (6.0091947655105145, 45.08956323641652, 1),
    52.09875800192703))),
 (u'OhmRdo9XL2ZWtpt7ewvPAg',
  ((u'Subway', u'Henderson', 36.0288225, -115.0624726),
   (u'Fast Food',
    (7.113644388284024, 45.23920582371423, 1),
    53.35285021199825)))]

In [68]:
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

businesses_test = sc.textFile('../YelpDataset/yelp_academic_dataset_business.json')    
    
def mapper_business_filter(rows):
    for row in rows:
        row = json.loads(row)
        for category in row['categories']:
            if(category in selectedCategories):
                if (len(row["full_address"].split("\n")) == 5):
                    if (RepresentsInt(row["full_address"].split("\n")[4].split(" ")[-1])):
                        yield (row['business_id'], row["full_address"].split("\n")[4].split(" ")[-1])
                        break
                elif (len(row["full_address"].split("\n")) == 4):
                    if (RepresentsInt(row["full_address"].split("\n")[3].split(" ")[-1])):
                        yield (row['business_id'], row["full_address"].split("\n")[3].split(" ")[-1])
                        break
                elif (len(row["full_address"].split("\n")) == 3):
                    if (RepresentsInt(row["full_address"].split("\n")[2].split(" ")[-1])):
                        yield (row['business_id'], row["full_address"].split("\n")[2].split(" ")[-1])
                        break
                elif (len(row["full_address"].split("\n")) == 2):
                    if (RepresentsInt(row["full_address"].split("\n")[1].split(" ")[-1])):
                        yield (row['business_id'], row["full_address"].split("\n")[1].split(" ")[-1]) 
                        break
                elif (len(row["full_address"].split("\n")) == 1):
                    if (RepresentsInt(row["full_address"].split("\n")[0].split(" ")[-1])):
                        yield (row['business_id'], row["full_address"].split("\n")[0].split(" ")[-1])
                        break

business_zip_filtered = businesses_test.mapPartitions(mapper_business_filter)

In [70]:
import csv

# business_information_joined = business_information_joined.collect()
# cities = ['LasVegas', 'Phoenix', 'Montreal', 'Edinburgh', 'Karlsruhe', 'Waterloo', 'Charlotte', 'Urbana', 'Madison', 'Champaign']
# cities = Set(cities)

def mapper_business_information(rows):
    for row in rows:
        row = json.loads(row)
        yield(row['business_id'], (row['name'], row['city'], row['latitude'], row['longitude']))

score = total_normalized_checkins.join(total_normalized_reviews).join(total_normalized_tips)
score = score.map(lambda x: (x[0][1], ((x[0][0], (x[1][0][0], x[1][0][1], x[1][1]), x[1][0][0] + x[1][0][1] + x[1][1]))))
business_information = businesses.mapPartitions(mapper_business_information)
business_information_joined = business_information.join(score)
business_information_joined.take(2)

business_zipCode_Joined = business_information_joined.join(business_zip_filtered)
business_zipCode_Joined.take(5)

business_zipCode_Joined_collect = business_zipCode_Joined.collect()

with open('data_zipcodes.csv', 'w') as csvfile:
    fieldnames = ['business_name', 'category', 'longitude', 'latitude', 'city', 'zip', 'total_score', 'country']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for business in business_zipCode_Joined_collect:
        writer.writerow({'business_name': str(business[1][0][0][0].encode('utf-8')), 'category': str(business[1][0][1][0]), \
                        'city': str(business[1][0][0][1].encode('utf-8')), 'longitude': str(business[1][0][0][2]), \
                        'zip': str(business[1][1]), 'latitude': str(business[1][0][0][3]), 'total_score': str(business[1][0][1][2]), \
                         'country': "United States" })
        

In [31]:
def mapper_zip_score(records):
    for row in records:
        yield (row[0], sum(row[1])/len(row[1]))
        
zips = business_zipCode_Joined.map(lambda x: ((x[1][0][0][1], x[1][1]), x[1][0][1][2])).groupByKey().mapPartitions(mapper_zip_score)
zips.take(10)

[((u'Phoenix', u'85033'), 53.976885547466885),
 ((u'Firth of Forth', 'NA'), 48.228258864525515),
 ((u'Edinburgh', u'4'), 64.78035825160369),
 ((u'Charlotte', u'28280'), 64.33979410937042),
 ((u'Litchfield Park', u'85340'), 57.81611154198246),
 ((u'Surprise', u'85379'), 60.42466032331306),
 ((u'Charlotte', u'28244'), 64.32240844273008),
 ((u'Laveen', u'85339'), 53.01440148421943),
 ((u'Belmont', u'28012'), 59.685557805864626),
 ((u'Saint-Henri', 'NA'), 64.05292636132184)]

In [43]:
k = range(20)

In [19]:
import csv
selectedCategories = ['Restaurants', 'Shopping', 'Beauty & Spas', 'Bars', 'Automotive', 'Event Planning & Services',
                     'Fast Food', 'Coffee & Tea', 'Hotels', 'Real Estate', 'Dentists', 'Gyms']

All = []

Restaurants = []
Shopping = []
BeautyNSpas = []
Bars = []
Automotive = []
EventPlanningNServices = []
FastFood = []
CoffeeNTea = []
Hotels = []
RealEstate = []
Dentists = []
Gyms = []

with open('final_data.csv', 'rb') as csvfile:
    reader = csv.reader(csvfile, quotechar='|')
    for row in reader:
        if row[0].startswith("business_name"): continue
        if row[1] == 'Restaurants':
            Restaurants.append(row)
        elif row[1] == 'Shopping':
            Shopping.append(row)
        elif row[1] == 'Beauty & Spas':
            BeautyNSpas.append(row)
        elif row[1] == 'Bars':
            Bars.append(row)
        elif row[1] == 'Automotive':
            Automotive.append(row)
        elif row[1] == 'Event Planning & Services':
            EventPlanningNServices.append(row)
        elif row[1] == 'Fast Food':
            FastFood.append(row)
        elif row[1] == 'Coffee & Tea':
            CoffeeNTea.append(row)
        elif row[1] == 'Hotels':
            Hotels.append(row)
        elif row[1] == 'Real Estate':
            RealEstate.append(row)
        elif row[1] == 'Dentists':
            Dentists.append(row)
        elif row[1] == 'Gyms':
            Gyms.append(row)

print len(Restaurants) + len(Shopping) + len(BeautyNSpas) + len(Bars) + len(Automotive) + len(EventPlanningNServices) + \
        len(FastFood) + len(CoffeeNTea) + len(Hotels) + len(RealEstate) + len(Dentists) + len (Gyms)


38353


In [20]:
All.append(Restaurants)
All.append(Shopping)
All.append(BeautyNSpas)
All.append(Bars)
All.append(Automotive)
All.append(EventPlanningNServices)
All.append(FastFood)
All.append(CoffeeNTea)
All.append(Hotels)
All.append(RealEstate)
All.append(Dentists)
All.append(Gyms)

In [21]:
for i in All:
    print i[0][1]
    with open(str(i[0][1]) + '.csv', 'wb') as f:
        writer = csv.writer(f)
        writer.writerow(['business_name', 'category', 'longitude', 'latitude', 'city', 'zip', 'total_score', 'country', 'state'])
        for j in i:
            writer.writerow(j)

Restaurants
Shopping
Beauty & Spas
Bars
Automotive
Event Planning & Services
Fast Food
Coffee & Tea
Hotels
Real Estate
Dentists
Gyms
